In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = 'SimHei'

road_distance_data = pd.read_csv('road_distance_data.csv')
borrow_data = pd.DataFrame({
    '楼宇名称': ['充电站1', '北辰楼', '秋水楼', '皓月楼', '雅望楼', '都青楼',
                 '四美楼', '弥津楼', '临川楼', '曾翠楼', '朝晖楼', '星耀楼',
                 '芳邻楼', '映雪楼', '十字路口', '朱华楼', '流丹楼', '万千楼',
                 '凌云楼', '高洁楼', '图书馆', 'T型路口', '俊采楼', '东隅楼',
                 '远山楼', '南架楼', '景明楼', '绿竹楼', '汀兰楼', '长风楼',
                 '重巽楼', '长洲楼', '清风楼', '沉壁楼', '兰亭楼', '充电站2'],
    '权重': [10, 12, 5, 8, 6, 4, 11, 9, 10, 7, 6, 9, 10, 11, 5, 8, 6, 4, 12, 9, 10, 8, 6, 9, 10, 12, 5, 8, 6, 4, 11, 9,
             10, 7, 6, 9]
})


def build_transposed_graph(borrow_data, road_data):
    G = nx.Graph()

    for idx, row in borrow_data.iterrows():
        G.add_node(row['楼宇名称'], weight=row['权重'])

    nodes = borrow_data['楼宇名称'].tolist()
    road_counter = 0

    for j in range(6):
        for i in range(5):
            start_node = nodes[i * 6 + j]
            end_node = nodes[(i + 1) * 6 + j]
            road_name = road_data.iloc[road_counter]['道路名称']
            road_length = road_data.iloc[road_counter]['实际距离']
            G.add_edge(start_node, end_node, road=road_name, length=road_length)
            road_counter += 1

    for i in range(6):
        for j in range(5):
            start_node = nodes[i * 6 + j]
            end_node = nodes[i * 6 + j + 1]
            road_name = road_data.iloc[road_counter]['道路名称']
            road_length = road_data.iloc[road_counter]['实际距离']
            G.add_edge(start_node, end_node, road=road_name, length=road_length)
            road_counter += 1

    return G


G_transposed = build_transposed_graph(borrow_data, road_distance_data)
pos = nx.spring_layout(G_transposed)
pos

nodes = borrow_data['楼宇名称'].tolist()
transposed_positions = [(j, i) for i in range(6) for j in range(6)]
transposed_node_positions = dict(zip(nodes, transposed_positions))
transposed_pos = {node: (y, -x) for node, (x, y) in transposed_node_positions.items()}
edge_labels_transposed = nx.get_edge_attributes(G_transposed, 'road')

edge_labels_with_distance = {(u, v): f"{d['road']} ({d['length']} km)" for u, v, d in G_transposed.edges(data=True)}

node_labels = {node: f"{node}\n权重: {G_transposed.nodes[node]['weight']}" for node in G_transposed.nodes()}

plt.figure(figsize=(15, 9))
nx.draw(G_transposed, transposed_pos, labels=node_labels, with_labels=True, font_weight='bold', node_size=3000,
        node_color='skyblue')
nx.draw_networkx_edge_labels(G_transposed, transposed_pos, edge_labels=edge_labels_with_distance, font_color='red')

plt.show()
# 计算 PageRank 和中介中心性
pagerank_values = nx.pagerank(G_transposed, weight='length')
betweenness_centrality_values = nx.betweenness_centrality(G_transposed, weight='length')

# 将结果添加到图的节点属性中
nx.set_node_attributes(G_transposed, pagerank_values, 'pagerank')
nx.set_node_attributes(G_transposed, betweenness_centrality_values, 'betweenness')
import numpy as np
import random


# 定义 AttractRank 算法
def attract_rank(G, alpha=0.5, beta=0.3, gamma=0.2):
    attractiveness = {}

    for node in G.nodes:
        pagerank_score = G.nodes[node]['pagerank']
        betweenness_score = G.nodes[node]['betweenness']
        node_weight = G.nodes[node]['weight']
        # 计算 AttractRank 分数，alpha, beta, gamma 是权重因子
        attract_rank_score = alpha * pagerank_score + beta * betweenness_score + gamma * node_weight
        attractiveness[node] = attract_rank_score

    return attractiveness


# 计算 AttractRank 值
attract_rank_values = attract_rank(G_transposed)

# 将 AttractRank 结果添加到图的节点属性中
nx.set_node_attributes(G_transposed, attract_rank_values, 'attract_rank')


# 结合道路长度和AttractRank值来计算新的权重
def custom_weight(u, v, d):
    # AttractRank越高，路径越有吸引力，d['length']为道路长度
    return d['length'] / (G_transposed.nodes[v]['attract_rank'] + 1e-6)


import threading
import time
import queue


class Car:
    def __init__(self, car_num, speed, start_position, end_position, path):
        self.car_num = car_num
        self.speed = speed
        self.start_position = start_position
        self.end_position = end_position
        self.path = path
        self.relative_time = 0.0

    def add_path_point(self, coords, travel_time):
        self.relative_time += travel_time
        self.path.append({
            'coords': coords,
            'relative_time': self.relative_time,
            'travel_time': travel_time,
            'timestamp': time.time()
        })

    # 定义比较方法
    def __lt__(self, other):
        # 以车辆路径的最后一个点的时间戳作为比较标准
        return self.path[-1]['timestamp'] < other.path[-1]['timestamp']


# 计算停留时间的函数
def calculate_stay_time(attractiveness):
    return max(0.1, attractiveness * 0.1)  # 最小停留时间为0.1小时


# 模拟车辆路径并行运行
def simulate_vehicle_path(G, road_data, pos, car_num, output_queue):
    uniform_speed = np.random.uniform(0.3, 0.7)  # 每辆车有不同的速度
    round_num = 0
    while round_num < 5:
        round_num += 1
        start_road_index = np.random.choice(road_data.index)
        end_road_index = np.random.choice(road_data.index)

        while end_road_index == start_road_index:
            end_road_index = np.random.choice(road_data.index)

        start_road = road_data.iloc[start_road_index]
        end_road = road_data.iloc[end_road_index]
        start_road_name = start_road['道路名称']
        end_road_name = end_road['道路名称']
        start_position = np.random.uniform(0, start_road['实际距离'])
        end_position = np.random.uniform(0, end_road['实际距离'])

        start_connected_edges = [(u, v) for u, v, d in G.edges(data=True) if d['road'] == start_road_name]
        end_connected_edges = [(u, v) for u, v, d in G.edges(data=True) if d['road'] == end_road_name]

        if not start_connected_edges or not end_connected_edges:
            continue

        start_edge = random.choice(start_connected_edges)
        end_edge = random.choice(end_connected_edges)

        try:
            if start_position < end_position:
                path = nx.dijkstra_path(G, source=start_edge[0], target=end_edge[1], weight=custom_weight)
            else:
                path = nx.dijkstra_path(G, source=start_edge[1], target=end_edge[0], weight=custom_weight)
        except nx.NetworkXNoPath:
            continue

        car = Car(car_num, uniform_speed, pos[start_edge[0]], pos[end_edge[1]], [])

        # 计算从起始位置到第一个节点的时间
        first_leg_time = start_position / car.speed
        car.add_path_point(car.start_position, first_leg_time)

        # 模拟车辆在路径中的移动
        for i in range(len(path) - 1):
            u = path[i]
            v = path[i + 1]
            edge_data = G[u][v]
            travel_time = edge_data['length'] / car.speed  # 计算每段行驶时间

            # 车辆到达节点前，计算停留时间并停留
            stay_time = calculate_stay_time(G.nodes[v]['attract_rank'])
            car.add_path_point(pos[v], travel_time + stay_time)

            # 在车辆停留期间增加节点权重
            G.nodes[v]['weight'] += 1
            nx.set_node_attributes(G, attract_rank(G), 'attract_rank')

            # 动态更新路径
            if i < len(path) - 2:
                try:
                    new_path = nx.dijkstra_path(G, source=v, target=path[-1], weight=custom_weight)
                    if new_path != path[i + 1:]:
                        path = path[:i + 1] + new_path
                except nx.NetworkXNoPath:
                    break

            # 在车辆离开节点时，恢复节点权重
            G.nodes[v]['weight'] -= 1
            nx.set_node_attributes(G, attract_rank(G), 'attract_rank')

        # 计算从最后一个节点到终点位置的时间
        last_leg_time = end_position / car.speed
        car.add_path_point(pos[path[-1]], last_leg_time)

        # 将结果放入优先队列中，根据车辆的到达时间排序
        output_queue.put((car.path[-1]['timestamp'], car, round_num))

        # 引入随机延迟
        time.sleep(np.random.uniform(1, 3))


#打印输出队列中的结果
def print_results(output_queue):
    while True:
        timestamp, car, round_num = output_queue.get()
        if car is None:  # 结束信号
            break
        print(f"车辆 {car.car_num} 在第 {round_num} 轮到达终点:")
        print(f"  起始点位置: {car.start_position}")
        print(f"  结束点位置: {car.end_position}")
        for path_point in car.path:
            timestamp_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(path_point['timestamp']))
            print(
                f"  到达坐标: {path_point['coords']}, 相对时间: {path_point['relative_time']:.2f}小时, 行驶时间: {path_point['travel_time']:.2f}小时, 时间戳: {timestamp_str}")
        print()


# 创建优先队列用于存储输出
output_queue = queue.PriorityQueue()
# 启动打印线程
print_thread = threading.Thread(target=print_results, args=(output_queue,))
print_thread.start()
# 为每辆车创建一个线程并开始模拟
threads = []
for car_num in range(1, 11):
    t = threading.Thread(target=simulate_vehicle_path,
                         args=(G_transposed, road_distance_data, pos, car_num, output_queue))
    threads.append(t)
    t.start()

# 等待所有车辆线程完成
for t in threads:
    t.join()

# 发送结束信号
output_queue.put((float('inf'), None, None))
# 等待打印线程完成
print_thread.join()
